In [2]:
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import pandas as pd
import numpy as np

In [3]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = AutoModel.from_pretrained("facebook/bart-base")

c:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
image_labels = pd.read_pickle('EEG_raw.pkl')['labels']
image_labels

0       goldfish, Carassius auratus
1       goldfish, Carassius auratus
2       goldfish, Carassius auratus
3       goldfish, Carassius auratus
4       goldfish, Carassius auratus
                   ...             
7198                    scuba diver
7199                    scuba diver
7200                    scuba diver
7201                    scuba diver
7202                    scuba diver
Name: labels, Length: 7203, dtype: object

In [6]:
def get_embeddings(words:str):
    input = tokenizer(words, return_tensors="pt")
    output = model(**input)
    last_hidden_state = output.last_hidden_state.detach().numpy().squeeze().mean(axis=0)
    return last_hidden_state

In [7]:
word_embeddings_df = np.empty((len(image_labels),768))
for i,image_label in tqdm(enumerate(image_labels)):
    synsets = [synset.strip().lower() for synset in image_label.split(',')]
    embeddings_sum = np.zeros(768)
    for synset in synsets:
        embeddings_sum += get_embeddings(synset)
    embeddings_sum /= len(synsets)
    word_embeddings_df[i,:] = embeddings_sum

7203it [14:40,  8.18it/s]


In [8]:
np.save('word_embeddings.npy',word_embeddings_df)